In [1]:
import os

# 현재 작업 디렉토리 출력
print(os.getcwd())

# 작업 디렉토리 변경
os.chdir('/home/ghpf/Yoon/2023 sep. 비SCI/Computer vision/Data/cucumber')

# 변경된 디렉토리 확인
print(os.getcwd())

/home/ghpf/Yoon/2023 sep. 비SCI/Computer vision/Data
/home/ghpf/Yoon/2023 sep. 비SCI/Computer vision/Data/cucumber


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 이미지 로드
image = cv2.imread("cucumber1.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# RGB에서 HSV로 변환
hsv_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

# 노란색의 HSV 범위를 정의
lower_yellow = np.array([25, 150, 150])
upper_yellow = np.array([35, 255, 255])

# HSV 이미지에서 노란색만 추출하여 바이너리 이미지 생성
yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

# 원본 이미지와 마스크를 비트 연산
yellow_flower = cv2.bitwise_and(image_rgb, image_rgb, mask=yellow_mask)

# HSV 이미지를 다시 RGB로 변환
hsv_in_rgb = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)

# ROI 지정 (x, y, width, height)
x, y, w, h = 800, 1200, 1500, 1000

# 빨간색으로 ROI 그리기
cv2.rectangle(hsv_in_rgb, (x, y), (x+w, y+h), (255, 0, 0), 5)
cv2.rectangle(yellow_flower, (x, y), (x+w, y+h), (255, 0, 0), 5)

# 노란색 꽃을 검출
contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 최소 면적 임계값 설정
min_area_threshold = 500

# 큰 면적을 가진 contours만 필터링
large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area_threshold]
flower_count = len(large_contours)

# 이미지 출력
plt.figure(figsize=(24, 5))

plt.subplot(1, 4, 1)
plt.imshow(image_rgb)
#plt.title('Original Image')

plt.subplot(1, 4, 2)
plt.imshow(hsv_in_rgb)
#plt.title('HSV Image with Red ROI Overlay')

plt.subplot(1, 4, 3)
plt.imshow(yellow_flower)
#plt.title(f'Detected Yellow Flower with Red ROI (Count: {flower_count})')

plt.subplot(1, 4, 4)
plt.imshow(yellow_mask, cmap='gray')
#plt.title('Binary Image of Detected Yellow Flower')

plt.tight_layout()

plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 이미지 로드
image = cv2.imread("cucumber1.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
hsv_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

# 노란색의 HSV 범위를 정의
lower_yellow = np.array([25, 150, 150])
upper_yellow = np.array([35, 255, 255])

# HSV 이미지에서 노란색만 추출하여 바이너리 이미지 생성
yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

# V 채널에서 깊이 정보를 가져옵니다.
depth_map = hsv_image[:, :, 2]

# 깊이 맵에서 노란색 마스크를 사용하여 깊이 값을 가져옵니다.
masked_depth = cv2.bitwise_and(depth_map, depth_map, mask=yellow_mask)

# 노란색 꽃의 contours를 찾습니다.
contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 각 꽃의 깊이 값을 평가하고 면적 값을 함께 저장합니다.
contour_scores = []
for cnt in contours:
    mask = np.zeros_like(yellow_mask)
    cv2.drawContours(mask, [cnt], 0, 255, thickness=cv2.FILLED)
    
    x_, y_, w_, h_ = cv2.boundingRect(cnt)
    if x_ > x and x_ + w_ < x + w and y_ > y and y_ + h_ < y + h:  # ROI 안에 있는 꽃만 고려합니다.
        cnt_depth = np.mean(masked_depth[mask == 255])
        cnt_area = cv2.contourArea(cnt)
        
        score = cnt_area / (1 + cnt_depth)
        contour_scores.append((cnt, score))

# 평가 점수에 따라 contours를 정렬합니다.
sorted_contours = sorted(contour_scores, key=lambda x: x[1], reverse=True)

# 평가 점수가 가장 높은 5개의 꽃 contours를 선택합니다.
top_five_contours = [cnt[0] for cnt in sorted_contours[:7]]

# 가장 높은 평가 점수를 가진 5개의 꽃을 빨간색으로 표시합니다.
output_image = image_rgb.copy()
for cnt in top_five_contours:
    cv2.drawContours(output_image, [cnt], 0, (255, 0, 0), 3)

# 원본 이미지에서 ROI 부분을 확대합니다.
original_roi = image_rgb[y:y+h, x:x+w]

# 빨간색으로 표시된 이미지에서 ROI 부분을 확대합니다.
output_roi = output_image[y:y+h, x:x+w]

# 이미지 출력
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(image_rgb)
#plt.title('Original Image')

plt.subplot(2, 2, 2)
plt.imshow(original_roi)
#plt.title('Zoomed ROI from Original Image')

plt.subplot(2, 2, 3)
plt.imshow(output_image)
#plt.title('Detected Top 5 Yellow Flowers')

plt.subplot(2, 2, 4)
plt.imshow(output_roi)
#plt.title('Zoomed ROI with Detected Flowers')

plt.tight_layout()
plt.savefig("cucumber_CV_flower.png", bbox_inches = 'tight', dpi=300)
plt.show()
